In [1]:
from Levenshtein import distance
import numpy as np
import pandas as pd
import os
import unicodedata
import re

In [2]:
def get_levenshtein(x,y):
    # ordering the inputs by length so that a<=b
    a=max(len(x), len(y))
    b=min(len(x), len(y))
    if a==0: # one of the input is empty
        return 1.
    if a > b+500: # if there are more than 500 more chars then return 100%
        return 1.
    # return the levenshtein distance normalized by the mean length
    mean_length = (a + b)/2.
    return 2 * distance(x,y)/mean_length

def normalize_text(x, to_upper = True):
    ans = unicodedata.normalize("NFKD", x.replace(";"," "))
    if(to_upper):
        return ans.upper()
    return ans
    

In [80]:
def keep_digits(x):
    return ''.join(filter(lambda x: x.isdigit(), x))

In [42]:
class Amendement:
    def __init__(self, txt, start_signal, start_offset, expose_signal):
        
        self.full_txt = txt
        self.full_txt_normalized = [normalize_text(t) for t in txt]
        self.start_signal = normalize_text(start_signal)
        self.expose_signal = normalize_text(expose_signal)
        self.start_offset = start_offset
        
        self.numero_amendement = self.get_amendement_nb()
        self.numero_article = self.get_amendement_article()
        self.txt_normalized, self.txt = self.get_amendement_text()
        
        
    def get_amendement_nb(self):
        x = self.full_txt_normalized
        for i in range(0, len(x)):
            if x[i][0:3]=="N° ":
                return x[i].replace("N° ", "")
        return "numéro amendement introuvable 0"
    
    def get_amendement_article(self):
        has_started=False
        x = self.full_txt_normalized
        for i in range(0, len(x)):
            if x[i]==self.start_signal:
                has_started = True

            if(has_started) and 'ARTICLE' in x[i]:
                return re.sub(".*ARTICLE ","",x[i])
        return ""
    
    def get_amendement_text(self):
        x = self.full_txt_normalized
        y = self.full_txt
        start,end=0,0
        
        expose_length = len(self.expose_signal)
        
        for i in range(0, len(x)):
            if x[i]==self.start_signal:
                start=i+self.start_offset

            if i<len(x)-3 and len(x[i])==0 and len(x[i+1])==0 and len(x[i+2])==0:
                end=i

            if len(x[i])<18 and x[i][0:expose_length]==self.expose_signal:
                end=i
                break
        candidate_normalized = x[start:end]
        candidate_raw = y[start:end]
        res_normalized, res_raw =[], []
        for i in range(0, len(candidate_normalized)):
            if normalize_text("Cet amendement est en cours") in candidate_normalized[i]:
                continue
            elif len(candidate_normalized[i]) < 2:
                continue
            else:
                res_normalized.append(candidate_normalized[i])
                res_raw.append(candidate_raw[i])
        return " ".join(res_normalized).replace("  ", " "), \
               " ".join(res_raw).replace("  ", " ")


In [67]:
class Liasse:
    def __init__(self, folder, projet_loi, start_offset, amendement_start_signal, amendement_start_offset_after_signal,\
                amendement_expose_signal): 
            self.folder = folder
            self.projet_loi = normalize_text(projet_loi)
            self.nb_amendements = 0
            
            starts=[]
            row=0
            all_txt, all_txt_normalized = [], []
            print("Lecture des fichiers dans {} : ".format(folder))
            for f in os.listdir(folder):
                print("{}".format(f), end=',')
                current_file= open(folder+"/"+f, encoding="iso-8859-1")
        
                for line in current_file:
                    all_txt.append(normalize_text(line.strip(), False))
                    all_txt_normalized.append(normalize_text(line.strip()))
                    if normalize_text(line.strip()[0:len(projet_loi)]) == self.projet_loi:
                        starts.append(row-start_offset)
                    row+=1

                current_file.close()
            print("")
            print("")

            self.all_txt = all_txt
            
            
            begins, ends, amendements = [],[],[]
            amendements_nb, amendements_txt, amendements_txt_normalized, amendements_art = [], [], [], []
            
            for i in range(0, len(starts)):
                if i<len(starts)-1:
                    begin=starts[i]
                    end=starts[i+1]
                else:
                    begin=starts[-1]
                    end=len(all_txt)

                current_amendement = Amendement(all_txt[begin:end], amendement_start_signal, \
                                                amendement_start_offset_after_signal, amendement_expose_signal)
                
                begins.append(begin)
                ends.append(end)
                amendements.append(current_amendement)
                
                amendements_nb.append(current_amendement.numero_amendement)
                amendements_txt.append(current_amendement.txt)
                amendements_txt_normalized.append(current_amendement.txt_normalized)
                amendements_art.append(current_amendement.numero_article)
                
            self.begins = begins    
            self.ends = ends
            self.amendements = amendements

            self.df_amendements = pd.DataFrame({'nb':amendements_nb, \
                                          'txt_brut':amendements_txt, \
                                          'txt':amendements_txt_normalized, \
                                          'article':amendements_art, 'begin':begins, 'end':ends})
            print("Il y a {} amendements dans la liasse {}".format(len(self.df_amendements), self.folder))
            

In [68]:
liasse_AN = Liasse("AN/", "EQUILIBRE DANS LE SECTEUR AGRICOLE ET ALIMENTAIRE", 5, \
                  "----------", 3, "EXPOS")

Lecture des fichiers dans AN/ : 
10_ apräs 15.txt,11_art 15 bis Ö avant 16.txt,12- art 16 Ö la fin.txt,1_  avant 11 et 11 liasse 2.txt,2 _ art 11- apräs 11.txt,3_ 11 bis apräs 11 quindecies.txt,4- art 11 sexdecies -apräs 12.txt,5_ art 12 bis -apräs 13 ter.txt,6_ apräs 13 ter- apräs 14.txt,8_ 14 bis -apräs 14 undecies.txt,9_ art 15.txt,

Il y a 1379 amendements dans la liasse AN/


In [69]:
liasse_senat = Liasse("senat/", "Relations commerciales dans le secteur agricole et alimentaire", 2, \
                  "présenté par", 9, "Objet")

Lecture des fichiers dans senat/ : 
.DS_Store,liasse sénat 1- 362 COMECO.txt,

Il y a 259 amendements dans la liasse senat/


In [70]:
liasse_senat.df_amendements.head()

,article,begin,end,nb,txt,txt_brut
0,15,3,39,COM-1,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,Insérer un article additionnel ainsi rédigé...
1,15 BIS (NOUVEAU),39,69,COM-2,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,Insérer un article additionnel ainsi rédigé...
2,11 SEXIES (NOUVEAU),69,123,COM-3,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,Insérer un article additionnel ainsi rédigé...
3,14 QUATER (NOUVEAU),123,149,COM-4,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,Insérer un article additionnel ainsi rédigé...
4,14 QUATER (NOUVEAU),149,183,COM-5,INSÉRER UN ARTICLE ADDITIONNEL AINSI RÉDIGÉ...,Insérer un article additionnel ainsi rédigé...


In [95]:
def compute_match(df_amendements_1, df_amendements_2, version1, version2):
    
    threshold = 0.01

    corresp={}

    for i1, row1 in df_amendements_1.iterrows():
        
        #if i1 > 30:
        #    break

        if i1 % 20 == 0:
            print( "{} %".format(int(i1 * 100 / len(df_amendements_1))), end="  ")
        for i2,row2 in df_amendements_2.iterrows():
            l = get_levenshtein(row1.txt,row2.txt)
            current_similarity = 1-l

            if(current_similarity > threshold):

                update = True
                if row2.nb in corresp:
                    previous_similarity = corresp[row2.nb][3]
                    if previous_similarity >= current_similarity:
                        update = False

                if update:
                    corresp[row2.nb] = (row1.nb, row1.txt_brut, row2.txt_brut, current_similarity, row2.article)
                    
    amendement_id2, amendement_id1, amendement_txt2, amendement_txt1, similarity, article2 \
    = [], [], [], [], [], []

    for id2 in corresp:
        id1, txt1, txt2, simi, art2 = corresp[id2] 
        amendement_id2.append(str(id2))
        amendement_id1.append(str(id1))
        amendement_txt2.append(txt2)
        article2.append(art2)
        amendement_txt1.append(txt1)
        similarity.append("{0:.0%}".format(simi))

    df_amendements_matched = pd.DataFrame( { \
                                         "N° amendement "+version1:amendement_id1, \
                                         "N° amendement "+version2:amendement_id2, \
                                         "Amendement "+version1:amendement_txt1, \
                                         "Amendement "+version2:amendement_txt2, \
                                         "N° article de la loi":article2, \
                                         "Similarité":similarity })
    print("Il y a {} amendements communs".format(len(df_amendements_matched)))
    
    df_amendements_matched["id2"] = df_amendements_matched["N° amendement "+version2]\
    .apply(lambda x:keep_digits(x))\
    .astype(int)
    df_amendements_matched.sort_values(by="id2", inplace = True, ascending=True)
    del df_amendements_matched["id2"]
    
    
    nb_id2_matched = len(df_amendements_matched["N° amendement "+version2].unique())
    nb_id1_matched = len(df_amendements_matched["N° amendement "+version1].unique())
    nb_id2 = len(df_amendements_2)
    nb_id1 = len(df_amendements_1)
    print("""Sur les {} amendements de la nouvelle liasse, {} ont été retrouvés dans la première liasse, \
    soit {:.0%}.
    Sur les {} amendements de l'ancienne liasse, {} ont été ré-utilisés dans la seconde liasse, \
    soit {:.0%}.
          """\
         .format(nb_id2, nb_id2_matched, nb_id2_matched/nb_id2, nb_id1, nb_id1_matched, nb_id1_matched/nb_id1))
    
    return df_amendements_matched

In [96]:
def match_amendement(df_amendements_2, df_amendements_matched, version1, version2):
    
    df_results = pd.merge(df_amendements_2, df_amendements_matched, left_on = "nb", right_on="N° amendement "+version2)

    df_results["N° amendement "+version2] = df_results["nb"]
    df_results["Amendement "+version2] = df_results["txt_brut"]

    del df_results["begin"], df_results["end"], df_results["nb"],  df_results["txt"],\
    df_results["txt_brut"], df_results["article"]

    df_results["N° amendement "+version1].fillna("non trouvé", inplace = True)
    df_results["Amendement "+version1].fillna("non trouvé", inplace = True)
    df_results["Similarité"].fillna("0%", inplace = True)
    
    df_results.to_excel("matching_{}_{}.xlsx".format(version1, version2), index=False)
    
    return df_results

mot="CUNICOLE"

df_amendements_1["cont"] = df_amendements_1["txt"].apply(lambda x:mot in x)

df_amendements_2["cont"] = df_amendements_2["txt"].apply(lambda x:mot in x)

df_amendements_1[df_amendements_1.cont==1]
df_amendements_2[df_amendements_2.cont==1]
del df_amendements_1["cont"]
del df_amendements_2["cont"]

In [97]:
df_amendements_matched = compute_match(liasse_AN.df_amendements, liasse_senat.df_amendements, "AN", "senat")
df_amendements_matched.head()

0 %  1 %  2 %  4 %  5 %  7 %  8 %  10 %  11 %  13 %  14 %  15 %  17 %  18 %  20 %  21 %  23 %  24 %  26 %  27 %  29 %  30 %  31 %  33 %  34 %  36 %  37 %  39 %  40 %  42 %  43 %  44 %  46 %  47 %  49 %  50 %  52 %  53 %  55 %  56 %  58 %  59 %  60 %  62 %  63 %  65 %  66 %  68 %  69 %  71 %  72 %  73 %  75 %  76 %  78 %  79 %  81 %  82 %  84 %  85 %  87 %  88 %  89 %  91 %  92 %  94 %  95 %  97 %  98 %  Il y a 110 amendements communs
Sur les 259 amendements de la nouvelle liasse, 110 ont été retrouvés dans la première liasse,     soit 42%.
    Sur les 1379 amendements de l'ancienne liasse, 87 ont été ré-utilisés dans la seconde liasse,     soit 6%.
          


,Amendement AN,Amendement senat,N° amendement AN,N° amendement senat,N° article de la loi,Similarité
1,A la fin du I. de l'article L 251-1 du code ru...,Insérer un article additionnel ainsi rédigé...,2058,COM-4,14 QUATER (NOUVEAU),62%
71,"APRÈS L'ARTICLE 15, insérer l'article suivan...",Après l'article 10 octies (nouveau) Insérer ...,76,COM-16,10 OCTIES (NOUVEAU),82%
102,La section 2 du chapitre Ier du titre Ier du l...,Avant l'article 16 A (nouveau) Insérer un art...,499,COM-18,16 A (NOUVEAU),8%
43,"« À partir dun cadre national, les régions ...",Après l'article 16 A (nouveau) Insérer un ar...,2317,COM-19,16 A (NOUVEAU),26%
99,Le titre II du Livre V de la partie législati...,Après l'article 11 quindecies (nouveau) Insé...,2131,COM-22,11 QUINDECIES (NOUVEAU),56%


In [99]:
df_results = match_amendement(liasse_senat.df_amendements, df_amendements_matched, "AN", "senat")
df_results.head()

,Amendement AN,Amendement senat,N° amendement AN,N° amendement senat,N° article de la loi,Similarité
0,A la fin du I. de l'article L 251-1 du code ru...,Insérer un article additionnel ainsi rédigé...,2058,COM-4,14 QUATER (NOUVEAU),62%
1,"APRÈS L'ARTICLE 15, insérer l'article suivan...",Après l'article 10 octies (nouveau) Insérer ...,76,COM-16,10 OCTIES (NOUVEAU),82%
2,La section 2 du chapitre Ier du titre Ier du l...,Avant l'article 16 A (nouveau) Insérer un art...,499,COM-18,16 A (NOUVEAU),8%
3,"« À partir dun cadre national, les régions ...",Après l'article 16 A (nouveau) Insérer un ar...,2317,COM-19,16 A (NOUVEAU),26%
4,Le titre II du Livre V de la partie législati...,Après l'article 11 quindecies (nouveau) Insé...,2131,COM-22,11 QUINDECIES (NOUVEAU),56%


In [ ]:
#%matplotlib inline
#df_amendements_matched["Simi"] = df_amendements_matched["Similarité"]\
#.apply(lambda x:int(x.replace("%", "")))
#df_amendements_matched.Simi.hist(bins=100)
#del df_amendements_matched["Simi"]

In [24]:
df_avis = pd.read_excel("tableau_sp.xls", skiprows=[0])[["N°", "Avis SAJ titre I"]].fillna("")

In [28]:
df_results_final = pd.merge(df_results, df_avis, left_on="N° AN", right_on="N°", how='left')
del df_results_final["N°"]
df_results_final.to_csv("amendements_communs_AN_senat.csv", index=False)
df_results_final.to_excel("amendements_communs_AN_senat.xls", index=False)
df_results_final.head()
print(len(df_results_final))

110
